In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=0.05))

In [9]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

In [10]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [11]:
run_logdir = get_run_logdir() 

In [14]:

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [16]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
tf.config.run_functions_eagerly(True)
history = model.fit(X_train, y_train, epochs=30,
 validation_data=(X_valid, y_valid),
 callbacks=[tensorboard_cb])

C:\Users\Dhruv\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


Epoch 1/30
  1/363 [..............................] - ETA: 0s - loss: 3.1832WARNING:tensorflow:From C:\Users\Dhruv\anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
363/363 [==============================] - 7s 20ms/step - loss: 0.5262 - val_loss: 171.4775
Epoch 2/30
363/363 [==============================] - 6s 17ms/step - loss: nan - val_loss: nan
Epoch 3/30
363/363 [==============================] - 6s 17ms/step - loss: nan - val_loss: nan
Epoch 4/30
363/363 [==============================] - 6s 18ms/step - loss: nan - val_loss: nan
Epoch 5/30
363/363 [==============================] - 6s 17ms/step - loss: nan - val_loss: nan
Epoch 6/30
363/363 [==============================] - 6s 17ms/step - loss: nan - val_loss: nan
Epoch 7/30
363/363 [==============================] - 6s 17ms/step - loss: 

In [17]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

In [20]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    options = {"input_shape": input_shape}
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", **options))
        options = {}
    model.add(keras.layers.Dense(1, **options))
    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [21]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [22]:
keras_reg.fit(X_train, y_train, epochs=100,
 validation_data=(X_valid, y_valid),
 callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 3s 9ms/step - loss: 1.1199 - val_loss: 10.5492
Epoch 2/100
363/363 [==============================] - 3s 9ms/step - loss: 0.6678 - val_loss: 0.5672
Epoch 3/100
363/363 [==============================] - 3s 9ms/step - loss: 0.5200 - val_loss: 0.4875
Epoch 4/100
363/363 [==============================] - 4s 10ms/step - loss: 0.4855 - val_loss: 0.4654
Epoch 5/100
363/363 [==============================] - 3s 9ms/step - loss: 0.4632 - val_loss: 0.4563
Epoch 6/100
363/363 [==============================] - 3s 9ms/step - loss: 0.4478 - val_loss: 0.4691
Epoch 7/100
363/363 [==============================] - 3s 9ms/step - loss: 0.4367 - val_loss: 0.4507
Epoch 8/100
363/363 [==============================] - 3s 8ms/step - loss: 0.4288 - val_loss: 0.4346
Epoch 9/100
363/363 [==============================] - 3s 9ms/step - loss: 0.4220 - val_loss: 0.4450
Epoch 10/100
363/363 [==============================] - 3s 10ms/step - loss: 0.4169 - val

In [24]:
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_test)

162/162 [==============================] - 1s 4ms/step - loss: 0.3567


In [25]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
param_distribs = {
 "n_hidden": [0, 1, 2, 3],
 "n_neurons": np.arange(1, 100),
 "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
 validation_data=(X_valid, y_valid),
 callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
  1/242 [..............................] - ETA: 0s - loss: 8.1177

C:\Users\Dhruv\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


242/242 [==============================] - 2s 10ms/step - loss: 3.8537 - val_loss: 1.5221
Epoch 2/100
242/242 [==============================] - 2s 9ms/step - loss: 1.1080 - val_loss: 0.7390
Epoch 3/100
242/242 [==============================] - 2s 8ms/step - loss: 0.6773 - val_loss: 0.6179
Epoch 4/100
242/242 [==============================] - 2s 8ms/step - loss: 0.5898 - val_loss: 0.5781
Epoch 5/100
242/242 [==============================] - 2s 9ms/step - loss: 0.5680 - val_loss: 0.5662
Epoch 6/100
242/242 [==============================] - 2s 9ms/step - loss: 0.5576 - val_loss: 0.7680
Epoch 7/100
242/242 [==============================] - 2s 8ms/step - loss: 0.5557 - val_loss: 0.7141
Epoch 8/100
242/242 [==============================] - 2s 8ms/step - loss: 0.5519 - val_loss: 0.7329
Epoch 9/100
242/242 [==============================] - 2s 9ms/step - loss: 0.5511 - val_loss: 0.5563
Epoch 10/100
242/242 [==============================] - 2s 10ms/step - loss: 0.5447 - val_loss: 0.7484

C:\Users\Dhruv\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


242/242 [==============================] - 2s 9ms/step - loss: 3.7034 - val_loss: 1.9653
Epoch 2/100
242/242 [==============================] - 2s 8ms/step - loss: 1.2683 - val_loss: 0.9443
Epoch 3/100
242/242 [==============================] - 2s 8ms/step - loss: 0.8153 - val_loss: 1.6347
Epoch 4/100
242/242 [==============================] - 2s 9ms/step - loss: 0.7012 - val_loss: 3.0315
Epoch 5/100
242/242 [==============================] - 2s 8ms/step - loss: 0.6572 - val_loss: 4.7697
Epoch 6/100
242/242 [==============================] - 2s 10ms/step - loss: 0.6312 - val_loss: 6.6540
Epoch 7/100
242/242 [==============================] - 2s 10ms/step - loss: 0.6118 - val_loss: 8.5159
Epoch 8/100
242/242 [==============================] - 3s 10ms/step - loss: 0.5961 - val_loss: 10.3407
Epoch 9/100
242/242 [==============================] - 2s 9ms/step - loss: 0.5829 - val_loss: 11.9862
Epoch 10/100
242/242 [==============================] - 2s 8ms/step - loss: 0.5718 - val_loss: 13.

...

C:\Users\Dhruv\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


242/242 [==============================] - 3s 14ms/step - loss: 1.1687 - val_loss: 11.5619
Epoch 2/100
242/242 [==============================] - 3s 14ms/step - loss: 0.6074 - val_loss: 2.7019
Epoch 3/100
242/242 [==============================] - 4s 15ms/step - loss: 0.5098 - val_loss: 0.5926
Epoch 4/100
242/242 [==============================] - 6s 25ms/step - loss: 0.4609 - val_loss: 0.4158
Epoch 5/100
242/242 [==============================] - 3s 14ms/step - loss: 0.4340 - val_loss: 0.3970
Epoch 6/100
242/242 [==============================] - 3s 13ms/step - loss: 0.4180 - val_loss: 0.3937
Epoch 7/100
242/242 [==============================] - 3s 14ms/step - loss: 0.4079 - val_loss: 0.4369
Epoch 8/100
242/242 [==============================] - 3s 13ms/step - loss: 0.3991 - val_loss: 0.5232
Epoch 9/100
242/242 [==============================] - 3s 13ms/step - loss: 0.3923 - val_loss: 0.6559
Epoch 10/100
242/242 [==============================] - 3s 13ms/step - loss: 0.3878 - val_los

C:\Users\Dhruv\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


242/242 [==============================] - 3s 13ms/step - loss: 1.3918 - val_loss: 0.7525
Epoch 2/100
242/242 [==============================] - 3s 13ms/step - loss: 0.6525 - val_loss: 0.5687
Epoch 3/100
242/242 [==============================] - 3s 13ms/step - loss: 0.5624 - val_loss: 0.5078
Epoch 4/100
242/242 [==============================] - 3s 14ms/step - loss: 0.5084 - val_loss: 0.5716
Epoch 5/100
242/242 [==============================] - 3s 13ms/step - loss: 0.4734 - val_loss: 0.5369
Epoch 6/100
242/242 [==============================] - 3s 13ms/step - loss: 0.4468 - val_loss: 0.5797
Epoch 7/100
242/242 [==============================] - 3s 13ms/step - loss: 0.4313 - val_loss: 0.5132
Epoch 8/100
242/242 [==============================] - 3s 12ms/step - loss: 0.4181 - val_loss: 0.4781
Epoch 9/100
242/242 [==============================] - 3s 11ms/step - loss: 0.4087 - val_loss: 0.3830
Epoch 10/100
242/242 [==============================] - 3s 13ms/step - loss: 0.4038 - val_loss

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x00000179E576D940>, as the constructor either does not set or modifies parameter learning_rate

In [ ]:
rnd_search_cv.best_params_